<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/chapter06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学習済みモデルによる画像分類

https://pystyle.info/pytorch-how-to-use-pretrained-model/

## 必要なモジュールのインポート

In [ ]:
import json
from pathlib import Path

import numpy as np
import torch
import torchvision
from PIL import Image
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets.utils import download_url

## デバイスの作成

In [ ]:
def get_device(use_gpu):
    if use_gpu and torch.cuda.is_available():
        # これを有効にしないと、計算した勾配が毎回異なり、再現性が担保できない。
        torch.backends.cudnn.deterministic = True
        return torch.device("cuda")
    else:
        return torch.device("cpu")


# デバイスを選択する。
device = get_device(use_gpu=True)

## モデルの作成

In [ ]:
model = torchvision.models.resnet50(pretrained=True).to(device)

## Transforms の作成

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # 標準化する。
    ]
)

## 画像の読み込み
画像は、ウィンドウ左端のファイルアイコンをクリックして現れるパネルにファイルをにドラッグ＆ドロップして下さい。

In [ ]:
img = Image.open("sample.jpg")
inputs = transform(img)
inputs = inputs.unsqueeze(0).to(device)

## 推論する

In [ ]:
model.eval()
outputs = model(inputs)

## 推論結果の解釈

In [ ]:
batch_probs = F.softmax(outputs, dim=1)
batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)

In [ ]:
def get_classes():
    if not Path("data/imagenet_class_index.json").exists():
        # ファイルが存在しない場合はダウンロードする。
        download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

    # クラス一覧を読み込む。
    with open("data/imagenet_class_index.json") as f:
        data = json.load(f)
        class_names = [x["ja"] for x in data]

    return class_names


# クラス名一覧を取得する。
class_names = get_classes()

### 確率の高い上位３クラスの表示

In [ ]:
for probs, indices in zip(batch_probs, batch_indices):
    for k in range(3):
        print(f"Top-{k + 1} {class_names[indices[k]]} {probs[k]:.2%}")

# Dataloader を使った推論
画面左のファイルパネルにあるdataフォルダ内にファイルを置くと複数の画像をまとめて推論できます。

In [ ]:
def _get_img_paths(img_dir):
    img_dir = Path(img_dir)
    img_extensions = [".jpg", ".jpeg", ".png", ".bmp"]
    img_paths = [str(p) for p in img_dir.iterdir() if p.suffix in img_extensions]
    img_paths.sort()

    return img_paths


class ImageFolder(Dataset):
    def __init__(self, img_dir, transform):
        # 画像ファイルのパス一覧を取得する。
        self.img_paths = _get_img_paths(img_dir)
        self.transform = transform

    def __getitem__(self, index):
        path = self.img_paths[index]
        img = Image.open(path)
        inputs = self.transform(img)

        return {"image": inputs, "path": path}

    def __len__(self):
        return len(self.img_paths)


# Dataset を作成する。
dataset = ImageFolder("data", transform)
# DataLoader を作成する。
dataloader = DataLoader(dataset, batch_size=8)

In [ ]:
from IPython import display

for batch in dataloader:
    inputs = batch["image"].to(device)
    outputs = model(inputs)

    batch_probs = F.softmax(outputs, dim=1)

    batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)

    for probs, indices, path in zip(batch_probs, batch_indices, batch["path"]):
        display.display(display.Image(path, width=224))
        print(f"path: {path}")
        for k in range(3):
            print(f"Top-{k + 1} {probs[k]:.2%} {class_names[indices[k]]}")

In [ ]:
from IPython import display

for batch in dataloader:
    inputs = batch["image"].to(device)
    outputs = model(inputs)

    batch_probs = F.softmax(outputs, dim=1)

    batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)

    for probs, indices, path in zip(batch_probs, batch_indices, batch["path"]):
        display.display(display.Image(path, width=224))
        print(f"path: {path}")
        for k in range(3):
            print(f"Top-{k + 1} {probs[k]:.2%} {class_names[indices[k]]}")